<a href="https://colab.research.google.com/github/JeongUgSeo/Aiffel_Exploration1/blob/main/Exploration1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image
import os, glob

print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


In [18]:
import os

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
#image_dir_path = os.getenv("HOME") + "content/Class3"
image_dir_path = '/content/Class3'
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

image_dir_path = '/content/Class2'
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

image_dir_path = '/content/Class1'
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

200  images to be resized.
200  images resized.
가위 이미지 resize 완료!
200  images to be resized.
200  images resized.
바위 이미지 resize 완료!
200  images to be resized.
200  images resized.
보 이미지 resize 완료!


In [22]:
import numpy as np

def load_data(img_path, number_of_data=600):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/Class3/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/Class2/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/Class1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = '/content'
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train_norm.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 600 입니다.
x_train shape: (600, 28, 28, 3)
y_train shape: (600,)


In [26]:
# 데이터 확인하기
print(x_train_norm.shape, y_train.shape)

# 데이터 나누기
from sklearn.model_selection import train_test_split

new_X_train, new_X_test, new_y_train, new_y_test = train_test_split(x_train_norm, 
                                                    y_train, 
                                                    test_size=0.2, 
                                                    random_state=42)

print(new_X_train.shape, new_y_train.shape)
print(new_X_test.shape, new_y_test.shape)

(600, 28, 28, 3) (600,)
(480, 28, 28, 3) (480,)
(120, 28, 28, 3) (120,)


In [31]:
# 모델 설계하기
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

Model에 추가된 Layer 개수:  7


In [32]:
# 모델 확인하기
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                25632     
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

In [35]:
# 학습하기
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(new_X_train, new_y_train, epochs=10)

Epoch 1/10
15/15 [==============================] - 1s 5ms/step - loss: 1.8227 - accuracy: 0.2917
Epoch 2/10
15/15 [==============================] - 0s 5ms/step - loss: 1.1323 - accuracy: 0.3979
Epoch 3/10
15/15 [==============================] - 0s 4ms/step - loss: 1.0474 - accuracy: 0.3938
Epoch 4/10
15/15 [==============================] - 0s 5ms/step - loss: 0.9957 - accuracy: 0.4771
Epoch 5/10
15/15 [==============================] - 0s 5ms/step - loss: 0.9402 - accuracy: 0.6479
Epoch 6/10
15/15 [==============================] - 0s 4ms/step - loss: 0.8187 - accuracy: 0.7563
Epoch 7/10
15/15 [==============================] - 0s 4ms/step - loss: 0.6926 - accuracy: 0.7625
Epoch 8/10
15/15 [==============================] - 0s 4ms/step - loss: 0.5337 - accuracy: 0.7812
Epoch 9/10
15/15 [==============================] - 0s 5ms/step - loss: 0.3952 - accuracy: 0.9146
Epoch 10/10
15/15 [==============================] - 0s 4ms/step - loss: 0.2912 - accuracy: 0.9646


In [36]:
test_loss, test_accuracy = model.evaluate(new_X_test,new_y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

4/4 - 0s - loss: 0.2447 - accuracy: 1.0000
test_loss: 0.24467802047729492 
test_accuracy: 1.0
